In [1]:
import gensim.downloader
from tqdm import tqdm
import string

In [2]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
glove_vec = gensim.downloader.load('glove-wiki-gigaword-200')

In [5]:
cosine_distance = glove_vec.similarity('man', 'women')
cosine_distance

0.4042789

In [6]:
cosine_distance = glove_vec.similarity('chair', 'throne')
cosine_distance

0.24691008

In [7]:
cosine_distance = glove_vec.similarity('water', 'baby')
cosine_distance

0.35296535

In [8]:
vec = glove_vec['king']-glove_vec['man']+glove_vec['women']

In [9]:
results = glove_vec.most_similar(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = results[0]
print(f'{most_similar_key} is to king as Woman is to Man.')

queen is to king as Woman is to Man.


In [10]:
result = glove_vec.most_similar(positive=['man', 'princess'], negative=['woman'])
most_similar_key, similarity = result[0]
print(f'{most_similar_key} is to princess as man is to woman.')

prince is to princess as man is to woman.


In [12]:
results = glove_vec.most_similar(positive=['child', 'woman'], negative=['adult'])
most_similar_key, similarity = results[0]
print(f'{most_similar_key} is to women as child is to adult.')

mother is to women as child is to adult.


In [13]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from google.colab import drive
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import pandas as pd

data = pd.read_csv('spam.csv', encoding='latin-1')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [15]:
# checking the distribution of the target variable
data["v1"].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [16]:
# pre processing text
def text_pre_processing(sms):

    # removing punctuations
    sms_wo_punctuations = [x for x in sms if x not in string.punctuation]
    sms_wo_punctuations = ''.join(sms_wo_punctuations)

    # keeping only english letters
    sms_wo_punctuations_only_words = [x for x in sms_wo_punctuations if x.isalpha() or x == " "]
    sms_wo_punctuations_only_words = ''.join(sms_wo_punctuations_only_words)

    # converting the data to lower case
    sms_wo_punctuations_only_words_lc = sms_wo_punctuations_only_words.lower()

    # removing stopwords
    sms_wo_punctuations_only_words_lc = sms_wo_punctuations_only_words.split(" ")
    sms_wo_punctuations_only_words_lc_wo_sw = [x for x in sms_wo_punctuations_only_words_lc if x not in stopwords.words('english')]
    sms_wo_punctuations_only_words_lc_wo_sw = ' '.join(sms_wo_punctuations_only_words_lc_wo_sw)

    return sms_wo_punctuations_only_words_lc_wo_sw

In [17]:
# pre processed sms
data_processed = pd.DataFrame(columns = ["pre_processed_sms"])
for i in tqdm(data["v2"]):
    data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
data_processed

Streaming output truncated to the last 5000 lines.
<ipython-input-17-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-17-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-17-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-17-cedb96c765ef>:4: FutureWarning: The frame.append method is deprecated and will be removed from panda

,pre_processed_sms
0,Go jurong point crazy Available bugis n great ...
1,Ok lar Joking wif u oni
2,Free entry wkly comp win FA Cup final tkts st...
3,U dun say early hor U c already say
4,Nah I dont think goes usf lives around though
...,...
5567,This nd time tried contact u U å Pound prize ...
5568,Will Ì b going esplanade fr home
5569,Pity mood Soany suggestions
5570,The guy bitching I acted like id interested bu...


In [18]:
# Splitting into train and test
x_train, x_test, y_train, y_test = train_test_split(data_processed, data["v1"], test_size = 0.20, random_state = 100)
x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [19]:
x_train

,pre_processed_sms
0,Ya nowonion roast
1,Sen told going join uncle finance cbe
2,Sorry Ill call later
3,You know So hows anthony Are bringing money Iv...
4,Huh Hyde park mel ah opps got confused Anyway ...
...,...
4452,I work mon thurs Sat cant leh Booked liao Whic...
4453,Ok knackered Just came home went sleep Not goo...
4454,Nah cant help Ive never iphone
4455,Its Still looking job How much Tas earn


In [38]:
words_in_sentences=[]
for i in range(len(x_train)):
    words_in_sentences.append(x_train['pre_processed_sms'].iloc[i])

In [39]:
# Using word to vec
algorit = Word2Vec( vector_size = 500, min_count = 1)
algorit.build_vocab(words_in_sentences)
algorit.train(words_in_sentences, total_examples = len(x_train), epochs = algorit.epochs)

(288666, 1246695)

In [40]:
# converting words to vectors
def convert_word_to_vector(sms, size):
    vec = np.zeros(500).reshape((1, 500))
    count = 0
    for word in sms:
        try:
            vec += algorit.wv.get_vector(word).reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [41]:
# converting train and test to vectors
x_train_vecs = scale(np.concatenate([convert_word_to_vector(x, 500) for x in x_train["pre_processed_sms"]]))
x_test_vecs = scale(np.concatenate([convert_word_to_vector(x, 500) for x in x_test["pre_processed_sms"]]))

In [42]:
from sklearn.preprocessing import MinMaxScaler

# Assuming X is your feature matrix
scaler = MinMaxScaler(feature_range=(0, 1))
x_train_vecs = np.concatenate([convert_word_to_vector(z, 500) for z in x_train["pre_processed_sms"]])
X_train_scaled = scaler.fit_transform(x_train_vecs)

In [43]:
# Train a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_scaled, y_train)

MultinomialNB()

In [44]:
from sklearn.metrics import accuracy_score
x_test_vecs = np.concatenate([convert_word_to_vector(x, 500) for x in x_test["pre_processed_sms"]])
X_test_scaled = scaler.fit_transform(x_test_vecs)

In [45]:
# train confusion matrix
y_train_pred = clf.predict(X_train_scaled)
accuracy_train = accuracy_score(y_train,y_train_pred)
accuracy_train

0.8763742427641912

In [31]:
confusion_matrix(y_train,y_train_pred)

array([[3588,  267],
       [ 286,  316]])

In [46]:
# test confusion matrix
y_test_pred = clf.predict(X_test_scaled)
accuracy_test = accuracy_score(y_test,y_test_pred)
accuracy_test

0.852914798206278

In [47]:
confusion_matrix(y_test,y_test_pred)

array([[851, 119],
       [ 45, 100]])